
# Audio degradation

This notebook downloads the `VoiceBank-DEMAND` data, and then apply degradation

## Passi:
1. Download the `VoiceBank-DEMAND` data.
2. Extract audio.
3. Load audio.
4. Create the dataset.



In [3]:
import os
import urllib.request
import zipfile

# Lista dei file da scaricare: nome logico → URL
files = {
    "clean_trainset": "https://datashare.ed.ac.uk/bitstream/handle/10283/2791/clean_trainset_28spk_wav.zip?sequence=2&isAllowed=y",
    "noisy_trainset": "https://datashare.ed.ac.uk/bitstream/handle/10283/2791/noisy_trainset_28spk_wav.zip?sequence=6&isAllowed=y",
    "clean_testset":  "https://datashare.ed.ac.uk/bitstream/handle/10283/2791/clean_testset_wav.zip?sequence=1&isAllowed=y",
    "noisy_testset":  "https://datashare.ed.ac.uk/bitstream/handle/10283/2791/noisy_testset_wav.zip?sequence=5&isAllowed=y"
}

# Percorso principale in cui salvare dataset e estrazione
base_path = "VoiceBank_DEMAND"
os.makedirs(base_path, exist_ok=True)

# Scarica e decomprimi ogni file
for name, url in files.items():
    zip_path = os.path.join(base_path, f"{name}.zip")
    extract_folder = os.path.join(base_path, name)

    # Scaricamento
    if not os.path.exists(zip_path):
        print(f"Scaricamento di {name}...")
        urllib.request.urlretrieve(url, zip_path)
        print(f"Scaricato {zip_path}")
    else:
        print(f"{zip_path} già presente, skip scaricamento.")

    # Estrazione
    if not os.path.exists(extract_folder):
        print(f"Estrazione di {name} in {extract_folder}...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_folder)
        print(f"Estratto in {extract_folder}")
    else:
        print(f"{extract_folder} già esistente, skip estrazione.")

print("\n✅ Tutti i file scaricati ed estratti correttamente.")


Scaricamento di clean_trainset...


KeyboardInterrupt: 

In [ ]:
import numpy as np
import librosa
import soundfile as sf
import os
import random

# Percorsi
extract_path = "fma_small"
output_path = "fma_pairs_siamese"
clean_path = os.path.join(output_path, "clean")
degraded_path = os.path.join(output_path, "degraded")

os.makedirs(clean_path, exist_ok=True)
os.makedirs(degraded_path, exist_ok=True)

# Funzioni di degradazione
def add_white_noise(audio, factor=None):
    if factor is None:
        factor = random.uniform(0.001, 0.01)
    noise = np.random.randn(len(audio))
    return audio + factor * noise

def add_clipping(audio, threshold=0.8):
    return np.clip(audio, -threshold, threshold)

def add_reverb(audio, sr):
    ir = np.zeros(int(sr * 0.3))
    ir[0] = 1.0
    ir[int(sr * 0.05)] = 0.6
    ir[int(sr * 0.1)] = 0.3
    return np.convolve(audio, ir, mode='full')[:len(audio)]

def change_pitch(audio, sr):
    if audio.ndim > 1:
        audio = librosa.to_mono(audio)
    n_steps = random.uniform(-2, 2)
    return librosa.effects.pitch_shift(audio, sr=sr, n_steps=n_steps)

def change_speed(audio, sr):
    if audio.ndim > 1:
        audio = librosa.to_mono(audio)
    rate = random.uniform(0.9, 1.1)
    return librosa.effects.time_stretch(audio, rate=rate)


# Lista delle trasformazioni
degradations = [
    lambda a, sr: add_white_noise(a),
    lambda a, sr: add_clipping(a),
    lambda a, sr: add_reverb(a, sr),
    lambda a, sr: change_pitch(a, sr),
    lambda a, sr: change_speed(a, sr)
]


# Generazione delle coppie (clean, degraded)
pair_index = 0
for root, dirs, files in os.walk(extract_path):
    for file in files:
        if file.endswith(".mp3"):
            file_path = os.path.join(root, file)
            try:
                audio, sr = librosa.load(file_path, sr=None, duration=10.0)
            except Exception as e:
                print(f"Errore nel file {file_path}: {e}")
                continue

            # Salva audio originale
            clean_file = os.path.join(clean_path, f"{pair_index}.wav")
            sf.write(clean_file, audio, sr)

            # Applica degradazione casuale
            degrade_fn = random.choice(degradations)
            try:
                degraded_audio = degrade_fn(audio, sr)
            except Exception as e:
                print(f"Errore nella degradazione di {file}: {e}")
                continue

            # Salva audio degradato
            degraded_file = os.path.join(degraded_path, f"{pair_index}.wav")
            sf.write(degraded_file, degraded_audio, sr)

            print(f"Coppia salvata: {pair_index}.wav")
            pair_index += 1


Coppia salvata: 0.wav
Coppia salvata: 1.wav
Coppia salvata: 2.wav
Coppia salvata: 3.wav
Coppia salvata: 4.wav
Coppia salvata: 5.wav
Coppia salvata: 6.wav
Coppia salvata: 7.wav
Coppia salvata: 8.wav
Coppia salvata: 9.wav
Coppia salvata: 10.wav
Coppia salvata: 11.wav
Coppia salvata: 12.wav
Coppia salvata: 13.wav
Coppia salvata: 14.wav
Coppia salvata: 15.wav
Coppia salvata: 16.wav
Coppia salvata: 17.wav
Coppia salvata: 18.wav
